In [29]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path
import xmltodict
from myfunctions import event_separation , image_crop, image_crop_negative, poi
import tifffile

## Input data from a folder ##

In [43]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\original_data'
images_dir = '221123_mtStayGold_U2OS_ZEISS_fl'
joined_path = os.path.join(files_dir, images_dir)
size=(2048,2048)
img,input_name,output_name,datacsv,pixel_size={},{},{},{},{}

for input_file in os.listdir(joined_path):
        joined_file_path = os.path.join(files_dir, images_dir, input_file)
        if '.tif' in input_file:
            tif  = tifffile.TiffFile(joined_file_path)
            mdInfoDict = xmltodict.parse(tif.ome_metadata, force_list={'Plane'})
            date, cell_type, dye, bf_fl, index  = input_file.split('_')
            number, ome, tiff= index.split('.')
            ##
            if int(number)==1 or int(number)==2 or int(number)==4 or int(number)==8:
                pixel_size_value=0
                pixel_size[int(number)-1]=1
            else:
                pixel_size_value=1
                pixel_size[int(number)-1]=0.103
            ##
            img[int(number)-1] = Image.open(joined_file_path)
            input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}'
            output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
            print('Loaded image:'+input_file)
        else:
            labels, index, bf_fl  = input_file.split('_')
            datacsv[int(index)-1] = pd.read_csv(joined_file_path)
            print('Loaded csv:'+ input_file)
print('Pixel scaling:',pixel_size)
max_number=int(len(datacsv))   

Loaded image:221123_U2OS_mitostaygold_GFP_1.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_10.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_11.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_12.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_2.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_3.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_4.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_5.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_6.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_7.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_8.ome.tif
Loaded image:221123_U2OS_mitostaygold_GFP_9.ome.tif
Loaded csv:labels_10_fl.csv
Loaded csv:labels_11_fl.csv
Loaded csv:labels_12_fl.csv
Loaded csv:labels_1_fl.csv
Loaded csv:labels_2_fl.csv
Loaded csv:labels_3_fl.csv
Loaded csv:labels_4_fl.csv
Loaded csv:labels_5_fl.csv
Loaded csv:labels_6_fl.csv
Loaded csv:labels_7_fl.csv
Loaded csv:labels_8_fl.csv
Loaded csv:labels_9_fl.csv
Pixel scaling: {0: 1, 9: 0.103, 10: 0.103, 11:

## Get all Gaussian Points of Interest ##

In [38]:
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    ##
    scale_value=float(pixel_size[x])
    print('scaling by',scale_value)
    print(csv['axis-1'][1])
    csv['axis-1']=csv['axis-1']*(1/scale_value)
    print(csv['axis-1'][1])
    csv['axis-2']=csv['axis-2']*(1/scale_value)
    ##
    for i in range(2,7,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        framenum=im.n_frames
        poi(csv,in_name,sigma,size,framenum)

scaling by 1.0
634.1531761796789
634.1531761796789


100%|██████████| 20/20 [00:04<00:00,  4.50it/s]


scaling by 1.0
1268.532745482331
1268.532745482331


100%|██████████| 16/16 [00:03<00:00,  4.37it/s]


scaling by 0.103
71.45433429848848
693.7314009561989


...

100%|██████████| 95/95 [00:20<00:00,  4.60it/s]


scaling by 0.103
35.16101984945009
341.36912475194265


100%|██████████| 27/27 [00:05<00:00,  4.51it/s]


scaling by 0.103
60.54719331110367
587.8368282631425


100%|██████████| 50/50 [00:11<00:00,  4.22it/s]


## Cut Images and Gaussian POIs ##

In [45]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_pos'
filepath= os.path.join(training_dir , folder_name)

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    print('file number:',x+1)
    ##
    # scale_value=float(pixel_size[x])
    # print('scaling by',scale_value)
    # print(csv['axis-1'][1])
    # csv['axis-1']=csv['axis-1']*(1/scale_value)
    # print(csv['axis-1'][1])
    # csv['axis-2']=csv['axis-2']*(1/scale_value)
    ##
    list1=event_separation(csv)
    l=len(list1)
    out_name=f'{output_name[x]}'
    image_crop(l,list1, csv, im,0, out_name,filepath)

    for i in range(2,7,1):
        sigma=(i,i)
        s=sigma[0]
        print('sigma:',s)
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        out_name_g=f'{output_name[x]}_sigma{s}'
        
        gauss=base_dir+f'\{in_name}'
        gauss_image=Image.open(gauss)
        image_crop(l,list1, csv, gauss_image,1, out_name_g,filepath)

file number: 1
scaling by 1.0
634.1531761796789
634.1531761796789
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 2
scaling by 1.0
1268.532745482331
1268.532745482331
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 3
scaling by 0.103
71.45433429848848
693.7314009561989
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 4
scaling by 1.0
798.452638600671
798.452638600671
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 5
scaling by 0.103
131.35021797124082
1275.244834672241
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 6
scaling by 0.103
165.78699733083386
1609.5824983576103
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 7
scaling by 0.103
107.17049726966415
1040.490264754021
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 8
scaling by 1.0
1647.0650363300715
1647.0650363300715
sigma: 2
sigma: 3
sigma: 4
sigma: 5
sigma: 6
file number: 9
scaling by 0.103
110.61643625126248
1073.945983021966
sigma: 2
sigma: 3
sigma: 4
sigm

## Cut Negative Frames ##

In [49]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_neg'
filepath= os.path.join(training_dir , folder_name)
from myfunctions import image_crop_negative

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    print('file number:',x)
    out_name=f'{output_name[x]}'

    list1=event_separation(csv)
    l=len(list1)
    image_crop_negative(l,list1, csv, im, out_name,filepath)

file number: 0
file number: 1
file number: 2
file number: 3
file number: 4
file number: 5
file number: 6
file number: 7
file number: 8
file number: 9
file number: 10
file number: 11
